In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import os

import requests
from PIL import Image
from io import BytesIO
from IPython.display import display

from typing import List
import torch

import chromadb

import shutil

import os
import datetime

from transformers import ViTImageProcessor, ViTModel, ViTImageProcessorFast
from PIL import Image
import requests

import pandas as pd
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb.utils.data_loaders import ImageLoader

/Users/shogun/.pyenv/versions/3.10.6/envs/inspiart/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Set the path to the file you'd like to load
file_name = "test_images_vector_df_csv.csv"
file_path = "/Users/shogun/code/gwen-m97/raw_data/test_images/"

# Load the latest version
test_image_vector_df = pd.read_csv(f"{file_path}{file_name}")

test_image_vector_df.head()

,Unnamed: 0,image,image_vector
0,0,fred,john
1,1,Paul_Cézanne_-_The_Basket_of_Apples_-_1926.252...,"tensor([[[-1.1763e-04, 1.3477e-02, -2.9715e-0..."
2,2,"Piet_Mondriaan,_1942_-_New_York_City_I.jpg","tensor([[[-0.0161, -0.0331, 0.1272, ..., -0...."
3,3,Two_Young_Girls_at_the_Piano_MET_rl1975.1.201....,"tensor([[[-0.0074, -0.1225, 0.0272, ..., -0...."
4,4,Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg,"tensor([[[ 0.0462, -0.0105, 0.0337, ..., 0...."


In [7]:
# Set the path to the file you'd like to load
file_name = "gwen_200_vector_df_csv_first.csv"
file_path = "/Users/shogun/code/gwen-m97/raw_data/gwen_200/"

# Load the latest version
gwen_200_vector_df_first = pd.read_csv(f"{file_path}{file_name}")

gwen_200_vector_df_first.head()

,Unnamed: 0,image,image_vector
0,0,fred,john
1,1,pierre-paul-prud-hon_louis-antoine-de-saint-ju...,"tensor([[[ 0.0779, -0.0746, -0.0250, ..., -0...."
2,2,pierre-paul-prud-hon_study-for-venus-and-adoni...,"tensor([[[ 0.0945, -0.1561, -0.1269, ..., -0...."
3,3,pierre-paul-prud-hon_untitled.jpg,"tensor([[[-0.1026, -0.0752, 0.0328, ..., -0...."
4,4,pyotr-konchalovsky_crimea-the-rocks-by-the-sea...,"tensor([[[-0.0983, -0.1852, -0.0368, ..., 0...."


In [8]:
# Set the path to the file you'd like to load
file_name = "gwen_200_vector_df_csv_second.csv"
file_path = "/Users/shogun/code/gwen-m97/raw_data/gwen_200/"

# Load the latest version
gwen_200_vector_df_second = pd.read_csv(f"{file_path}{file_name}")

gwen_200_vector_df_second.head()

,Unnamed: 0,image,image_vector
0,0,fred,john
1,1,abbott-handerson-thayer_blue-jays-in-winter-19...,"tensor([[[ 0.0911, -0.1870, -0.0223, ..., -0...."
2,2,albert-gleizes_untitled-2.jpg!Large.jpg,"tensor([[[-0.0461, -0.1976, -0.0997, ..., 0...."
3,3,alexander-shilov_rosemary-flowered-1980(4).jpg...,"tensor([[[ 0.0573, -0.0742, -0.0356, ..., -0...."
4,4,alexandre-jacovleff_nu-2.jpg!Large.jpg,"tensor([[[ 1.8831e-01, -2.9391e-01, 2.2604e-0..."


In [38]:
gwen_200_vector_df = pd.concat([gwen_200_vector_df_first,gwen_200_vector_df_second],ignore_index=True)
gwen_200_vector_df.head()

,Unnamed: 0,image,image_vector
0,0,fred,john
1,1,pierre-paul-prud-hon_louis-antoine-de-saint-ju...,"tensor([[[ 0.0779, -0.0746, -0.0250, ..., -0...."
2,2,pierre-paul-prud-hon_study-for-venus-and-adoni...,"tensor([[[ 0.0945, -0.1561, -0.1269, ..., -0...."
3,3,pierre-paul-prud-hon_untitled.jpg,"tensor([[[-0.1026, -0.0752, 0.0328, ..., -0...."
4,4,pyotr-konchalovsky_crimea-the-rocks-by-the-sea...,"tensor([[[-0.0983, -0.1852, -0.0368, ..., 0...."


In [39]:
gwen_200_vector_df.shape

(195, 3)

In [40]:
gwen_200_vector_df.dtypes

Unnamed: 0       int64
image           object
image_vector    object
dtype: object

In [66]:
gwen_200_vector_df['image_vector'].to_numpy()[1].rsplit("tensor(")[1].rsplit(",\n")[0].to_numpy()

AttributeError: 'str' object has no attribute 'to_numpy'

In [84]:
chroma browse images_collection_db --local


SyntaxError: invalid syntax (1338063953.py, line 1)

In [2]:
class GoogleVITHuge224Embedding(EmbeddingFunction):


    def __call__(self, input: Documents) -> Embeddings:

        from transformers import ViTImageProcessor, ViTModel, ViTImageProcessorFast

        #Instantiate the image. Convert it to 244 x 244 and normalise RGB between 0 and 1 witha mean of 0.5 for each channel

        self.feature_extractor = ViTImageProcessorFast.from_pretrained('google/vit-huge-patch14-224-in21k')

        #Instantiate the Google ViT with pretrained weights

        self.model = ViTModel.from_pretrained('google/vit-huge-patch14-224-in21k')#Preprocess the data

        inputs = self.feature_extractor(images=input, return_tensors="pt")

        #Embedd the data

        outputs = self.model(**inputs)

        #Convert the embedding to a Numpy array and take the first vector of the Transformer state

        embeddings = outputs.last_hidden_state.data.numpy()[0,0]

        print(embeddings)

        #return the embedding

        return embeddings

In [33]:
chroma_client = chromadb.PersistentClient(path='/Users/shogun/code/gwen-m97/inspiart/my_vectordb')


In [35]:
image_loader = ImageLoader()

image_embbeding_function = GoogleVITHuge224Embedding()


/var/folders/1r/626773hx3mbf2w_31tdjwv6m0000gn/T/ipykernel_73451/3517319003.py:3: DeprecationWarning: The class GoogleVITHuge224Embedding does not implement __init__. This will be required in a future version.
  image_embbeding_function = GoogleVITHuge224Embedding()


In [36]:
images_collection_db = chroma_client.get_or_create_collection(name="images_collection_db", embedding_function=image_embbeding_function, data_loader=image_loader)

In [40]:
images_collection_db.query(
    query_uris=['/Users/shogun/code/gwen-m97/raw_data/test_images/Paul_Cézanne_-_The_Basket_of_Apples_-_1926.252_-_Art_Institute_of_Chicago.jpg']
)

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 12520.31it/s]


[-1.9647181e-05  1.3060389e-02 -2.9653966e-01 ... -1.0215039e-01
  9.6137896e-02 -6.0683262e-02]


{'ids': [['156', '50', '114', '41', '67', '178', '172', '150', '159', '142']],
 'embeddings': None,
 'documents': [[None, None, None, None, None, None, None, None, None, None]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'url': '/Users/shogun/code/gwen-m97/raw_data/gwen_200/pyotr-konchalovsky_still-life-peaches-1935.jpg!Large.jpg',
    'image': 'pyotr-konchalovsky_still-life-peaches-1935.jpg!Large.jpg'},
   {'url': '/Users/shogun/code/gwen-m97/raw_data/gwen_200/ford-madox-brown_the-finding-of-don-juan-by-haid-e-1871.jpg',
    'image': 'ford-madox-brown_the-finding-of-don-juan-by-haid-e-1871.jpg'},
   {'image': 'le-pho_mother-and-child.jpg',
    'url': '/Users/shogun/code/gwen-m97/raw_data/gwen_200/le-pho_mother-and-child.jpg'},
   {'image': 'edouard-manet_the-blonde-with-bare-breasts-1878.jpg!Large.jpg',
    'url': '/Users/shogun/code/gwen-m97/raw_data/gwen_200/edouard-manet_the-blonde-with-bare-breasts-1878.jpg!Large.jpg'},
   

In [38]:
images_collection_db.get()

{'ids': ['id1',
  '0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62',
  '63',
  '64',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70',
  '71',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '79',
  '80',
  '81',
  '82',
  '83',
  '84',
  '85',
  '86',
  '87',
  '88',
  '89',
  '90',
  '91',
  '92',
  '93',
  '94',
  '95',
  '96',
  '97',
  '98',
  '99',
  '100',
  '101',
  '102',
  '103',
  '104',
  '105',
  '106',
  '107',
  '108',
  '109',
  '110',
  '111',
  '112',
  '113',
  '114',
  '115',
  '116',
  '117',
  '118',
  '119',
  '120',
  '12

In [39]:
images_collection_db.count()

194